In [1]:
import pysb.bng
import kholodenko_basic as kb
import kholodenko_m1 as m
#import kholodenko_m3 as m3 # this is just to test redundant rules
import sympy as sp
import sys
from sympy.printing import latex
import re
import numpy as np
import tabulate as tb
import pandas as pd
import collections
pysb.bng.__file__

'/home/priya/pysb/pysb/bng.pyc'

In [2]:
pysb.bng.generate_equations(kb.model)
pysb.bng.generate_equations(m.model)
#pysb.bng.generate_equations(m3.model)

In [3]:
"""for i, j in zip(kb.model.species, ke.model.species):
 
    assert i.is_equivalent_to(j)

species_map = dict(zip(range(len(kb.model.species)), kb.model.species))

ke_subs, kb_subs = [zip(m.model.expressions,[e.expand_expr(expand_observables=True) for e in m.model.expressions]) for m in ke, kb]
print ke.model.parameters[2]
ke_dict, kb_dict = [dict([((x['reactants'], x['products']), x['rate']) for x in y])
                                    for y in [ke.model.reactions, kb.model.reactions]]

"""
"""
TODO:
0. NOT DONE:
    organize by products and reactants then subset by rule name because rule names are NOT conserved
DONE:
1. Substitute the species __s0 with some identifier Done 23 oct
2. Find a list of all of the UNIQUE bidirectional reactions Done 23 oct"""

import pysb.bng
import sympy as sp
import sys
from sympy.printing import latex
import re
import numpy as np
import tabulate as tb
import pandas as pd
import collections
import kholodenko_basic as kb
import kholodenko_m1 as m

def fmt_rct(r, species_dict, fmt='plain'):
    
    arrow = {'latex': " $\longleftrightarrow$ ",
             'plain': "<-->"}[fmt]
    fmt_fct = {'latex': lambda x: "$%s$" % latex(x).replace("__", "").replace(" % ", ""),
               'plain': lambda x: str(x).replace(" % ", "")}[fmt]
    r_schema = " + ".join(map(fmt_fct, r.reactants)) + arrow + "".join(map(fmt_fct, r.products))
    return [r.name.replace("_", "-"), r_schema, fmt_fct(r.fwd_rate), fmt_fct(r.rev_rate), fmt_fct(r.kd)]


def gen_table(model_list, fmt='plain'):
    
    # this is assuming that all models must have the same species....
    m = model_list[0]
    species_dict = dict([('__s%d' % idx, str(m.model.species[idx])) for idx in range(len(m.model.species))])
    
    all_rates_tidy = pd.DataFrame(data=np.vstack([[[m.__name__.replace("_", "-")] \
                                                   + fmt_rct(x, species_dict, fmt=fmt) \
                                                   for x in gen_rct_rates(m)] \
                                                   for m in model_list]), \
                                columns=["model", "rule", "schema", "fwd_rate", "rev_rate", "kd"])

    return np.array(species_dict.items()),  all_rates_tidy.sort_values(["schema"])
        
def gen_rct_rates(m):
    ## from a model, create a generator of all reaction rates binned by forward, reverse, kd
  
    rct_subs = dict(zip(m.model.expressions,[e.expand_expr(expand_observables=True) for e in m.model.expressions])) #\+ [(sp.Symbol('__s%d' % m.model.get_species_index(complex_pattern=species)), latex(species)) for species in m.model.species])
    species_map = dict(zip(range(len(m.model.species)), m.model.species))
    species = lambda tup: [species_map[x] for x in tup]
    
    fwd_rxn = dict([((r["rule"][0], r["products"], r["reactants"]), r) for r in m.model.reactions if not r["reverse"][0]])
    rev_rxn = dict([((r["rule"][0], r["reactants"], r["products"]), r) for r in m.model.reactions if r["reverse"][0]])
    rxn_tup = collections.namedtuple("reaction", ["name", "reactants", "products", "fwd_rate", "rev_rate", "kd"])
    rxn_ids = fwd_rxn.keys()
    
    def make_rct_tuple(rxn_id):
        
        rule_name = rxn_id[0]
        rev_rate = extra_simple(rev_rxn[rxn_id]["rate"].subs(rct_subs))
        fwd_rate = extra_simple(fwd_rxn[rxn_id]["rate"].subs(rct_subs))
        kd = extra_simple(rev_rate/fwd_rate)
        reactants, products = [species(fwd_rxn[rxn_id][s]) for s in ["reactants", "products"]]

        return rxn_tup(rule_name, reactants, products, fwd_rate, rev_rate, kd)
    
    return map(make_rct_tuple, rxn_ids)


def extra_simple(mul):
    return sp.powsimp((sp.expand_power_base(sp.powdenest(sp.logcombine(sp.expand_log(mul.simplify(), force=True), force=True), force=True), force=True)), force=True)

def model_name(model):
    return model.__name__.replace("_", "-")
    
def table_to_printstream(species, table, fmt='plain'):
    table = table.sort_values("model")
    table = table[["model", "schema", "fwd_rate", "rev_rate", "kd"]].as_matrix()
    
    if fmt == 'latex':
        return "\n".join(["\documentclass{article}",
                             "\usepackage[utf8]{inputenc}", 
                             "\usepackage{amsmath}", 
                             "\\begin{document}", 
                            '\def\\arraystretch{2}',
                          tb.tabulate(species, tablefmt='latex_raw'),
                          tb.tabulate(table, tablefmt='latex_raw'),
                          "\end{document}"])
    elif fmt == 'plain':
        return "\n".join([tb.tabulate(species),
                          tb.tabulate(table)])
                         

def main():
    models = [m, kb]
    species, rxns = gen_table(models, fmt='plain')
    sys.stdout.write(table_to_printstream(species, rxns, fmt='plain'))


main()


----  -------------------------------------------
__s3  B(d=None, i=1) % I(b=1)
__s2  B(d=1, i=None) % B(d=1, i=None)
__s1  I(b=None)
__s0  B(d=None, i=None)
__s5  B(d=1, i=2) % B(d=1, i=3) % I(b=2) % I(b=3)
__s4  B(d=1, i=2) % B(d=1, i=None) % I(b=2)
----  -------------------------------------------
----------------  ---------------------------------------------------------------------------------  ------------------------------------------  ----------------------------------------  ---------------------------------------------
kholodenko-basic  B(d=None, i=1)I(b=1) + B(d=None, i=1)I(b=1)<-->B(d=1, i=2)B(d=1, i=3)I(b=2)I(b=3)  0.5*__s3**2*(f**(-1)*g**(-1))**phi*k_on_BB  __s5*(f*g)**(1 - phi)*k_off_BB            2.0*__s5*k_on_BB**(-1)*f*g*k_off_BB/__s3**2
kholodenko-basic  B(d=None, i=None) + B(d=None, i=1)I(b=1)<-->B(d=1, i=2)B(d=1, i=None)I(b=2)        __s0*__s3*f**(-phi)*k_on_BB                 __s4*f**(1 - phi)*k_off_BB                __s4*k_on_BB**(-1)*f*k_off_BB/(__s0*__s3)
kholo

### dir(m)

In [14]:
print str(m.model.name)
dir(m.model)


kholodenko_m1


['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_component_types',
 '_export',
 '_extra_expressions',
 '_odes',
 '_rename_component',
 '_stoichiometry_matrix',
 '_validate_initial_condition_pattern',
 'add_annotation',
 'add_component',
 'all_component_sets',
 'all_components',
 'annotations',
 'base',
 'compartments',
 'diffusivities',
 'enable_synth_deg',
 'energypatterns',
 'expressions',
 'expressions_constant',
 'expressions_dynamic',
 'get_annotations',
 'get_species_index',
 'has_synth_deg',
 'initial',
 'initial_conditions',
 'monomers',
 'name',
 'observables',
 'odes',
 'parameters',
 'parameters_compartments',
 'parameters_initial_conditions',
 'parameters_rules',
 'parameters_unused',
 'reactions',
 'reactions_bidirectional',
 'reload',


In [ ]:
import collections
g = collections.namedtuple("reaction", ["h", "i", "j"])
dir(g(2, 3, 5))